In [1]:
import cv2
import numpy as np
import ctypes
import random
import time
import winsound
import wave
import contextlib
import tkinter as tk
import os
from PIL import Image, ImageTk

In [2]:
##To load model from system

from keras.models import model_from_json

with open('model.json', 'r') as f:
    loaded_model_json = f.read()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [3]:
#Function to randomly select song from a list of songs
def selectSong():
    filename = random.choice(os.listdir("C:\\Users\\zoyas\\Desktop\\Dance-Arcade-main\\songs"))
   # filename = "deepurban.wav"
    return "C:\\Users\\zoyas\\Desktop\\Dance-Arcade-main\\songs\\"+filename

In [4]:
#Function to calculate calories burnt in 1 round
def calculateCalories(bodyWeight, duration):
    MET = 4.5    #considered as dancing
    calories = duration * MET * 3.5 * bodyWeight / (200 * 60)
    return calories

In [5]:
#Function to prepare image as required by model

def prepImg(pth):
    return cv2.resize(pth,(300, 350)).reshape(1, 300, 350, 3)


In [6]:
#Function to select a box at random
def select():
    a = random.randint(1, 5)
    return a

In [7]:
shape_to_label = {'foot':np.array([1., 0.]), 'not_foot':np.array([0., 1.])}
arr_to_shape = {np.argmax(shape_to_label[x]) : x for x in shape_to_label.keys()}

In [8]:
#Function to highlight box randomly selected

def highlight(boxNo, frame):
    if(boxNo == 1):
        cv2.rectangle(frame, (int(Width/2 -75), int(Height-150)), (int(Width/2 + 75), int(Height-50)), (255, 0, 0), 2)
    elif(boxNo == 2):
        cv2.rectangle(frame, (int(Width/2 -245), int(Height-150)), (int(Width/2 - 95), int(Height-50)), (255, 0, 0), 2)
    elif(boxNo == 3):
        cv2.rectangle(frame, (int(Width/2 +245), int(Height-150)), (int(Width/2 + 95), int(Height-50)), (255, 0, 0), 2)
    elif(boxNo == 4):
        cv2.rectangle(frame, (int(Width/2 -245), int(Height-270)), (int(Width/2 - 95), int(Height-170)), (255, 0, 0), 2)
    else:
        cv2.rectangle(frame, (int(Width/2 +245), int(Height-270)), (int(Width/2 +95), int(Height-170)), (255, 0, 0), 2)

In [9]:
#Function to crop highlighted box from frame

def cropFrame(boxNo, frame):
    Width = len(frame[0])
    Height = len(frame)
    
    if(boxNo == 1):
        cropImg = frame[Height - 150 : Height - 50, Width // 2 - 75 : Width // 2 + 75]
    elif(boxNo == 2):
        cropImg = frame[Height - 150 : Height - 50, Width // 2 - 245 : Width // 2 - 95]
    elif(boxNo == 3):
        cropImg = frame[Height - 150 : Height - 50, Width // 2 + 95 : Width // 2 + 245]
    elif(boxNo == 4):
        cropImg = frame[Height - 270 : Height - 170, Width // 2 - 245 : Width // 2 - 95]
    else:
        cropImg = frame[Height - 270 : Height - 170, Width // 2 + 95 : Width // 2 + 245]
        
    return cropImg

In [10]:
#Function to highlight boxes for duration of song

def play_game(start_time, duration, tile_color):
    rounds = 1
    score = 0
    
    while True:
    
        curr_time = time.time()
        elapsed_time = curr_time - start_time

        if(elapsed_time > duration / 4):
            break

        pred = ""
        rounds += 1
        gotFrame = False
        for i in range(30):
            ret, frame = cap.read()

            if frame is None:
                continue
            cv2.rectangle(frame, (int(Width//2 - 75), int(Height-150)), (int(Width//2 +75), int(Height-50)), tile_color, 2)#1
            cv2.rectangle(frame, (int(Width//2 - 245), int(Height-150)), (int(Width//2 - 95), int(Height-50)), tile_color, 2)#2
            cv2.rectangle(frame, (int(Width//2 + 245), int(Height-150)), (int(Width//2 +95), int(Height-50)), tile_color, 2)#3
            cv2.rectangle(frame, (int(Width//2 - 245), int(Height-270)), (int(Width//2 - 95), int(Height-170)), tile_color, 2)#4
            cv2.rectangle(frame, (int(Width//2 + 245), int(Height - 270)), (int(Width//2 +95), int(Height-170)), tile_color, 2)#5

            # Select random box   
            if i/10 == 1:
                boxNo = select()
                print("selecting box ", boxNo)

            #Highlighting Box
            elif i // 10 == 1 or i // 10 == 2:
                highlight(boxNo, frame)

            if(i/10 > 1.5 and i/10 < 2.2 and not gotFrame):
                if(ret):
                    print(rounds + 1, " - ", len(frame[0]), len(frame))
                    crop = cropFrame(boxNo, frame)
                    print("Cropped - ", len(crop[0]), len(crop))
                    newCrop = prepImg(crop)
                    prediction = (arr_to_shape[np.argmax(loaded_model.predict(newCrop))])
                    print(prediction)

                    if(prediction == 'foot'):
                        score = score + 1
                        gotFrame = True
                    #cv2.imshow("cropped", cropImg)

            frame = cv2.flip(frame, 1)

            cv2.imshow('captured', frame)

            if cv2.waitKey(1) & 0xff == ord('q'):
                break
            
    return score

In [11]:
def show_score(final_score, duration):
    score_img = cv2.imread("score_img2.jpg", cv2.IMREAD_COLOR)
    replay_img = cv2.imread("replay.jpeg", cv2.IMREAD_COLOR)
    exit_img = cv2.imread("exit.jpeg", cv2.IMREAD_COLOR)

    replay_img = cv2.resize(replay_img,(150,80))
    exit_img = cv2.resize(exit_img,(150,80))
    print(score_img.shape)

    calories = str(round(calculateCalories(float(weight), duration), 2)) + ' kcals'
    #cv2.waitKey(0)
    score_img = cv2.putText(score_img, "GAME OVER", (130, 180), cv2.FONT_HERSHEY_DUPLEX, 2, (128, 0, 128), 6, cv2.LINE_AA)
    score_img = cv2.putText(score_img, username + "'s Score " + " = {}".format(final_score), (160, 250), cv2.FONT_HERSHEY_TRIPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
    score_img = cv2.putText(score_img, "Calories Burnt = {}".format(calories), (60, 290), cv2.FONT_HERSHEY_TRIPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)

    # button dimensions (y1, y2, x1, x2)
    button = [20, 60, 50, 250]


    # create a window and attach a mousecallback and a trackbar
    cv2.setMouseCallback('captured', process_click)

    # create button image
    print(score_img.shape)
    score_img[320 : 320 + replay_img.shape[0], 100 : 100 + replay_img.shape[1]] = replay_img
    score_img[320 : 320 + exit_img.shape[0], 350 : 350 + exit_img.shape[1]] = exit_img

    cv2.imshow("captured", score_img)
    cv2.waitKey(0)


    cap.release()
    cv2.destroyAllWindows()

In [12]:
#Function to resize background image if window resizes

def on_resize(event):
    # resize the background image to the size of label
    image = bgimg.resize((event.width, event.height), Image.ANTIALIAS)
    # update the image of the label
    l.image = ImageTk.PhotoImage(image)
    l.config(image=l.image)

In [13]:
#Function to store user details on submission

def submit():
 
    name = name_var.get()
    weight = weight_var.get()
    color = tile_color_var.get()
     
    print("The name is : " + name)
    print("The weight is : " + weight)
    print("The tile color is : " + color)
    
    root.destroy()


In [14]:
def position_user(tile_color = (255, 255, 255)):
    while True:
        ret, frame = cap.read()
        
        frame = cv2.flip(frame, 1)
        frame = cv2.putText(frame, "Position yourself on the O", (230, 150), cv2.FONT_HERSHEY_DUPLEX, 2, (250, 250, 0), 5, cv2.LINE_AA)
        frame = cv2.putText(frame, "PRESS SPACE TO START", (240, 220), cv2.FONT_HERSHEY_DUPLEX, 2, (250, 250, 0), 5, cv2.LINE_AA)
        
        cv2.rectangle(frame, (int (Width // 2 - 75 ), int (Height- 150)), (int (Width // 2 + 75), int (Height - 50)), tile_color, 2)         #1
        cv2.rectangle(frame, (int (Width // 2 - 245), int (Height - 150)), (int (Width // 2 - 95), int (Height - 50)), tile_color, 2)        #2
        cv2.rectangle(frame, (int (Width // 2 + 245), int (Height - 150)), (int (Width // 2 + 95), int (Height - 50)), tile_color, 2)        #3
        cv2.rectangle(frame, (int (Width // 2 - 245), int (Height - 270)), (int (Width // 2 - 95), int (Height - 170)), tile_color, 2)       #4
        cv2.rectangle(frame, (int (Width // 2 + 245), int (Height - 270)), (int (Width // 2 + 95), int (Height - 170)), tile_color, 2)       #5
        cv2.circle(frame, (int (Width // 2),int(Height - 220)), radius = 30, color = (255, 0, 255), thickness = 3)
        cv2.imshow("captured", frame)

        if cv2.waitKey(1) & 0xFF == ord(' '):
            break

In [15]:
def start_game(tile_color = (255, 255, 255)):
    position_user(tile_color)
    
    start_countdown()
    
    songfile = selectSong()

    with contextlib.closing(wave.open(songfile,'r')) as f:
        frames = f.getnframes()
        rate = f.getframerate()
        duration = frames / float(rate)

    print('Song duration = ', int(duration))
    
    winsound.PlaySound(songfile, winsound.SND_ASYNC)

    start_time = time.time()
    
    final_score = play_game(start_time, duration, tile_color)
    
    winsound.PlaySound(None, winsound.SND_ASYNC)
    show_score(final_score, duration)
    
    return final_score, duration

In [16]:
def start_countdown(tile_color = (255, 255, 255)):
    for i in range(90, 0, -1):
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        
        cv2.rectangle(frame, (int (Width // 2 - 75 ), int (Height- 150)), (int (Width // 2 + 75), int (Height - 50)), tile_color, 2)         #1
        cv2.rectangle(frame, (int (Width // 2 - 245), int (Height - 150)), (int (Width // 2 - 95), int (Height - 50)), tile_color, 2)        #2
        cv2.rectangle(frame, (int (Width // 2 + 245), int (Height - 150)), (int (Width // 2 + 95), int (Height - 50)), tile_color, 2)        #3
        cv2.rectangle(frame, (int (Width // 2 - 245), int (Height - 270)), (int (Width // 2 - 95), int (Height - 170)), tile_color, 2)       #4
        cv2.rectangle(frame, (int (Width // 2 + 245), int (Height - 270)), (int (Width // 2 + 95), int (Height - 170)), tile_color, 2)       #5
        cv2.circle(frame, (int (Width // 2), int (Height - 220)), radius = 30, color = (255, 0, 255), thickness = 3)

        # Countdown    
        if i//20 < 3 :
            frame = cv2.putText(frame,"Get Ready In " + str(i // 20 + 1), (320, 100), cv2.FONT_HERSHEY_DUPLEX, 3, (4, 54, 84), 2, cv2.LINE_AA)

        cv2.imshow('captured', frame)
        if cv2.waitKey(1) & 0xff == ord('q'):
            break
        

In [20]:
# function that handles the mousclicks
def process_click(event, x, y, flags, params):
    # check if the click is within the dimensions of the button
    if event == cv2.EVENT_LBUTTONDOWN:
        #print("mouse clicked")
        if y > 320 and y < 400 and x > 100 and x < 250:   
            print('Clicked on replay!')
            start_game(tile_color)
            
        elif y > 320 and y < 400 and x > 350 and x < 500:   
            print('Clicked on exit!')
            cap.release()
            cv2.destroyAllWindows()

In [21]:
#Function to get user name, weight and preference

root = tk.Tk()
bgimg = None

name_var = tk.StringVar()
weight_var = tk.StringVar()
tile_color_var = tk.StringVar()

def get_user_details():
    

    root.attributes('-fullscreen', True)

    bgimg = Image.open('user_input_bg.jpg') # load the background image
    l = tk.Label(root)
    l.place(x=0, y=0, relwidth=1, relheight=1) # make label l to fit the parent window always
    l.bind('<Configure>', on_resize) # on_resize will be executed whenever label l is resized


    options = ['black', 'white']

    label = tk.Label(root, text="Enter Details", font=('Broadway', 35,'normal'), bg = 'dark turquoise')
    label.grid(row=1, column=1, padx = 10, pady = 45, columnspan = 2)

    label = tk.Label(root, text="Enter Player Name ", font=('Broadway', 17,'normal'), bg = 'medium violet red')
    label.grid(row=2, column=1, padx = 10, pady = 10)
    name_entry = tk.Entry(root, textvariable = name_var, font=('Baskerville Old Face', 18,'normal'))
    name_entry.grid(row=2, column=2, padx = 10, pady = 10)

    label2 = tk.Label(root, text="Enter Player Weight ", font=('Broadway', 17,'normal'), bg = 'medium violet red')
    label2.grid(row=4, column=1, padx = 10, pady = 10)
    weight_entry = tk.Entry(root, textvariable = weight_var, font=('Baskerville Old Face', 18,'normal'))
    weight_entry.grid(row=4, column=2, padx = 10, pady = 10)

    label3 = tk.Label(root, text = "Select Tile Border Color ", font=('Broadway', 17,'normal'), bg = 'medium violet red')
    label3.grid(row = 6, column = 1, padx = 10, pady = 10)
    tile_color_var.set("black")
    tile_color_select = tk.OptionMenu( root , tile_color_var, *options)
    tile_color_select.config(width = 15, height = 1, font = ('Baskerville Old Face', 18,'normal'), bg = 'white', fg = 'black')
    menu = root.nametowidget(tile_color_select.menuname)
    menu.config(font=('Baskerville Old Face', 15,'normal'))
    tile_color_select.grid(row = 6, column = 2, padx = 10, pady = 10)

    sub_btn=tk.Button(root, text = 'SUBMIT', command = submit, width = 20, height = 1, bg = 'medium sea green', font=('Bauhaus 93', 15,'normal'))
    sub_btn.grid(row = 7, column = 1, padx = 10, pady = 25, columnspan = 2)

    root.grid_rowconfigure(0, weight=1)
    root.grid_rowconfigure(8, weight=1)
    root.grid_columnconfigure(0, weight=1)
    root.grid_columnconfigure(8, weight=1)

    root.mainloop()
    
    username = name_var.get()
    weight = weight_var.get()
    tile_color = tile_color_var.get()
    
    return username, weight, tile_color

In [22]:
user32 = ctypes.windll.user32
screen_width, screen_height = user32.GetSystemMetrics(0), user32.GetSystemMetrics(1)

cap = cv2.VideoCapture(0)

cv2.namedWindow("captured", cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty("captured", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

cap.set(3, screen_width)
cap.set(4, screen_height)

ret, frame = cap.read()


Width = len(frame[0])
Height = len(frame)

NUM_ROUNDS = 5
boxNo = -1
gotFrame = False
score = 0
rounds = 0

username = ""
weight = None
colour = None

home_img = cv2.imread("background_text.jpg", cv2.IMREAD_COLOR)
cv2.imshow("captured", home_img)
cv2.waitKey(5000)
        
print(screen_width, screen_height)
print(len(frame), len(frame[0]))

#can play only .wav audio files


#final_score = play_game(start_time, duration)
username, weight, colour = get_user_details()

print('user details = ', username, weight, colour)

if(colour == "black"):
    tile_color = (0, 0, 0)
else:
    tile_color = (255, 255, 255)

final_score, duration = start_game(tile_color)
print('final score = ', final_score)



1280 720
720 1280


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\zoyas\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-12-da50d18105cf>", line 5, in on_resize
    image = bgimg.resize((event.width, event.height), Image.ANTIALIAS)
AttributeError: 'NoneType' object has no attribute 'resize'


The name is : zoya
The weight is : 60
The tile color is : white
user details =  zoya 60 white
Song duration =  288
selecting box  5
3  -  1280 720
Cropped -  150 100
not_foot
3  -  1280 720
Cropped -  150 100
not_foot
3  -  1280 720
Cropped -  150 100
not_foot
3  -  1280 720
Cropped -  150 100
not_foot
3  -  1280 720
Cropped -  150 100
not_foot
3  -  1280 720
Cropped -  150 100
not_foot
selecting box  5
4  -  1280 720
Cropped -  150 100
not_foot
4  -  1280 720
Cropped -  150 100
not_foot
4  -  1280 720
Cropped -  150 100
not_foot
4  -  1280 720
Cropped -  150 100
foot
selecting box  2
5  -  1280 720
Cropped -  150 100
not_foot
5  -  1280 720
Cropped -  150 100
not_foot
5  -  1280 720
Cropped -  150 100
not_foot
5  -  1280 720
Cropped -  150 100
foot
selecting box  1
6  -  1280 720
Cropped -  150 100
not_foot
6  -  1280 720
Cropped -  150 100
not_foot
6  -  1280 720
Cropped -  150 100
not_foot
6  -  1280 720
Cropped -  150 100
foot
selecting box  1
7  -  1280 720
Cropped -  150 100
foot